In [2]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 37.3 MB/s eta 0:00:00


In [3]:
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,      # Process each image independently
    max_num_hands=1,             # Detect at most one hand per image
    min_detection_confidence=0.5 # Adjust as needed
)

In [9]:
root_dir = "/content/drive/My Drive/asl_train_short"

In [10]:
data = []

In [12]:
# Loop over each subdirectory (class)
for class_name in os.listdir(root_dir):
    class_dir = os.path.join(root_dir, class_name)
    if os.path.isdir(class_dir):
        print(f"Processing class: {class_name}")
        # Loop over all images in the subdirectory
        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(class_dir, filename)
                image = cv2.imread(image_path)
                if image is None:
                    continue  # Skip unreadable images
                # Convert image from BGR to RGB as MediaPipe expects RGB input
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = hands.process(image_rgb)

                # Prepare a row for this image
                row = {"filename": image_path, "class": class_name}

                # If hand landmarks are detected, extract the 21 landmark coordinates
                if results.multi_hand_landmarks:
                    hand_landmarks = results.multi_hand_landmarks[0]
                    for i, landmark in enumerate(hand_landmarks.landmark):
                        row[f'lm_{i}_x'] = landmark.x
                        row[f'lm_{i}_y'] = landmark.y
                        row[f'lm_{i}_z'] = landmark.z
                else:
                    # If no hand is detected, fill with NaN values
                    for i in range(21):
                        row[f'lm_{i}_x'] = np.nan
                        row[f'lm_{i}_y'] = np.nan
                        row[f'lm_{i}_z'] = np.nan

                data.append(row)

# Release MediaPipe resources
hands.close()

Processing class: L
Processing class: W
Processing class: Y
Processing class: P
Processing class: Q
Processing class: X
Processing class: E
Processing class: K
Processing class: J
Processing class: V
Processing class: nothing
Processing class: del
Processing class: M
Processing class: space
Processing class: S
Processing class: F
Processing class: O
Processing class: H
Processing class: G
Processing class: Z
Processing class: T
Processing class: N
Processing class: U
Processing class: I
Processing class: R
Processing class: C
Processing class: A
Processing class: D
Processing class: B


In [13]:

# Convert collected data into a Pandas DataFrame
df = pd.DataFrame(data)
print("DataFrame shape:", df.shape)

# Save DataFrame to CSV file for later use
df.to_csv("asl_hand_landmarks.csv", index=False)

DataFrame shape: (7635, 65)


In [14]:
df.head()

,filename,class,lm_0_x,lm_0_y,lm_0_z,lm_1_x,lm_1_y,lm_1_z,lm_2_x,lm_2_y,...,lm_17_z,lm_18_x,lm_18_y,lm_18_z,lm_19_x,lm_19_y,lm_19_z,lm_20_x,lm_20_y,lm_20_z
0,/content/drive/My Drive/asl_train_short/D/D121...,D,0.511094,0.591716,-3.487474e-07,0.607051,0.529379,0.025570,0.653169,0.466511,...,-0.043650,0.511544,0.242165,-0.052381,0.514430,0.177381,-0.048034,0.509313,0.133697,-0.041167
1,/content/drive/My Drive/asl_train_short/D/D100...,D,0.329109,0.763733,1.877792e-07,0.433378,0.673404,0.040782,0.495831,0.601142,...,-0.067104,0.376194,0.376045,-0.089025,0.455050,0.385423,-0.082711,0.503668,0.421793,-0.071885
2,/content/drive/My Drive/asl_train_short/D/D137...,D,0.632936,0.560647,4.836424e-07,0.709376,0.513662,-0.008589,0.760105,0.460751,...,-0.018038,0.654791,0.294193,-0.045939,0.709544,0.300495,-0.053550,0.743223,0.333564,-0.051424
3,/content/drive/My Drive/asl_train_short/D/D101...,D,0.577178,0.898925,8.493101e-07,0.695876,0.813673,0.004485,0.768492,0.718042,...,-0.145337,0.701501,0.523018,-0.172641,0.785972,0.523326,-0.160071,0.835409,0.554788,-0.143774
4,/content/drive/My Drive/asl_train_short/D/D101...,D,0.600141,0.903777,8.337612e-07,0.717342,0.809177,0.005050,0.788599,0.719976,...,-0.146353,0.720110,0.529502,-0.175751,0.804969,0.528707,-0.164853,0.856913,0.555495,-0.149651
